<a href="https://colab.research.google.com/github/flaviohnm/projeto_bancodedados/blob/main/fator_contribuinte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Instalação do módulos

In [3]:
pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 8.6 MB/s 


#Bloco de Extração

In [4]:
import pandas as pd
import pandera as pa

In [6]:
df = pd.read_csv("/content/drive/MyDrive/CENIPA/fator_contribuinte.csv",sep=";",parse_dates=['fator_nome'],dayfirst=True)

## O comando "head" exibe a quantidade de linhas contida no parentesis.

In [7]:
df.head(10)

,codigo_ocorrencia3,fator_nome,fator_aspecto,fator_condicionante,fator_area
0,45331,APLICAÇÃO DE COMANDOS,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
1,45331,ATENÇÃO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
2,45331,CAPACITAÇÃO E TREINAMENTO,ASPECTO PSICOLÓGICO,ORGANIZACIONAL,FATOR HUMANO
3,45331,CLIMA ORGANIZACIONAL,ASPECTO PSICOLÓGICO,ORGANIZACIONAL,FATOR HUMANO
4,45331,COMUNICAÇÃO,ASPECTO PSICOLÓGICO,PSICOSSOCIAL,FATOR HUMANO
5,45331,COORDENAÇÃO DE CABINE,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
6,45331,CULTURA DO GRUPO DE TRABALHO,ASPECTO PSICOLÓGICO,PSICOSSOCIAL,FATOR HUMANO
7,45331,CULTURA ORGANIZACIONAL,ASPECTO PSICOLÓGICO,ORGANIZACIONAL,FATOR HUMANO
8,45331,DESVIO DE NAVEGAÇÃO,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
9,45331,DINÂMICA DE EQUIPE,ASPECTO PSICOLÓGICO,PSICOSSOCIAL,FATOR HUMANO


Exibir os tipos de dados dos objetos

In [8]:
df.dtypes

codigo_ocorrencia3      int64
fator_nome             object
fator_aspecto          object
fator_condicionante    object
fator_area             object
dtype: object

Até esse trecho foi realizado o carregamento de dados com a conversão das colunas que contém datas para o tipo de dado "date" com o método "parse".

In [9]:
df.replace(['***','**','####','###!','****','*****','NULL'], pd.NA, inplace=True)

In [14]:
schema = pa.DataFrameSchema(
    columns = {
        "fator_nome":pa.Column(pa.String),
        "fator_aspecto": pa.Column((pa.String),nullable=True),
        "fator_condicionante":pa.Column((pa.String),nullable=True),
        "fator_area":pa.Column((pa.String),nullable=True)
    }
)

In [15]:
df.isnull().sum()

codigo_ocorrencia3       0
fator_nome               0
fator_aspecto           19
fator_condicionante    354
fator_area              19
dtype: int64

##Validação do Schema

In [16]:
schema.validate(df)

,codigo_ocorrencia3,fator_nome,fator_aspecto,fator_condicionante,fator_area
0,45331,APLICAÇÃO DE COMANDOS,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
1,45331,ATENÇÃO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
2,45331,CAPACITAÇÃO E TREINAMENTO,ASPECTO PSICOLÓGICO,ORGANIZACIONAL,FATOR HUMANO
3,45331,CLIMA ORGANIZACIONAL,ASPECTO PSICOLÓGICO,ORGANIZACIONAL,FATOR HUMANO
4,45331,COMUNICAÇÃO,ASPECTO PSICOLÓGICO,PSICOSSOCIAL,FATOR HUMANO
...,...,...,...,...,...
3459,79713,SUPERVISÃO GERENCIAL,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
3460,79729,OUTRO FATOR,OUTRO,<NA>,OUTRO
3461,80073,MANUTENÇÃO DA AERONAVE,DESEMPENHO DO SER HUMANO,MANUTENÇÃO DA AERONAVE,FATOR OPERACIONAL
3462,80258,INFRAESTRUTURA AEROPORTUÁRIA,INFRAESTRUTURA AEROPORTUÁRIA,<NA>,FATOR OPERACIONAL


##Checando se o valor da coluna "codigo ocorrência" são únicos.

In [18]:
df.codigo_ocorrencia3.is_unique

False

Definindo "codigo da ocorrência" como indice

In [20]:
df.set_index('codigo_ocorrencia3', inplace=True)

Consultar quantos NA tem no dataframe

In [21]:
df.isna().sum()

fator_nome               0
fator_aspecto           19
fator_condicionante    354
fator_area              19
dtype: int64

In [22]:
df.isnull().sum()

fator_nome               0
fator_aspecto           19
fator_condicionante    354
fator_area              19
dtype: int64